## POS tagging using modified Viterbi

### Data Preparation

In [1]:
#Importing libraries
import nltk, re, pprint
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import pprint, time
import random
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [2]:
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

In [3]:
# Splitting into train and test
random.seed(1234)
train_set, test_set = train_test_split(nltk_data,test_size=0.05)

In [4]:
# Getting list of tagged words
train_tagged_words = [tup for sent in train_set for tup in sent]
# tokens 
tokens = [pair[0] for pair in train_tagged_words]
# vocabulary
V = set(tokens)
# number of tags
T = set([pair[1] for pair in train_tagged_words])

### Build the vanilla Viterbi based POS tagger

In [5]:
#Emission Probabilities
# computing P(w/t) and storing in T x V matrix
t = len(T)
v = len(V)
w_given_t = np.zeros((t, v))

In [6]:
# compute word given tag: Emission Probability
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)
    
    return (count_w_given_tag, count_tag)

In [7]:
# compute tag given tag: tag2(t2) given tag1 (t1), i.e. Transition Probability

def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [8]:
# creating t x t transition matrix of tags
# each column is t2, each row is t1
# thus M(i, j) represents P(tj given ti)

tags_matrix = np.zeros((len(T), len(T)), dtype='float32')
for i, t1 in enumerate(list(T)):
    for j, t2 in enumerate(list(T)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

In [9]:
# convert the matrix to a df for better readability
tags_df = pd.DataFrame(tags_matrix, columns = list(T), index=list(T))

In [10]:
tags_df.loc['.', :]

VERB    0.088410
CONJ    0.057862
ADP     0.090207
NUM     0.081132
DET     0.172866
.       0.094429
X       0.026235
PRT     0.002336
ADV     0.052561
ADJ     0.044564
NOUN    0.223360
PRON    0.065948
Name: ., dtype: float32

In [11]:
# Viterbi Heuristic
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = sorted(list(set([pair[1] for pair in train_bag])))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [12]:
# Running on entire 5% test dataset.
# Assigning 5% test_set to test_run

test_run = test_set

# list of tagged words
test_run_base = [tup for sent in test_run for tup in sent]

# list of untagged words
test_tagged_words = [tup[0] for sent in test_run for tup in sent]

In [13]:
tagged_seq = Viterbi(test_tagged_words)

In [35]:
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 

accuracy = len(check)/len(tagged_seq)

accuracy

0.9050608081600627

## Solve the problem of unknown words

## Technique 1 Rule Based 

In [15]:
#To improve the performance,we specify a rule base tagger for unknown words 
# specify patterns for tagging
patterns = [
    (r'.*ing$', 'VERB'),              # gerund
    (r'.*ed$', 'VERB'),               # past tense 
    (r'.*es$', 'VERB'),               # verb    
    (r'.*\'s$', 'NOUN'),              # possessive nouns
    (r'.*s$', 'NOUN'),                # plural nouns
    (r'\*T?\*?-[0-9]+$', 'X'),        # X
    (r'^-?[0-9]+(.[0-9]+)?$', 'NUM'), # cardinal numbers
    (r'.*', 'NOUN')                   # nouns
]
 
# Assigning regex patterns for rule based tagger
rule_based_tagger = nltk.RegexpTagger(patterns)

In [16]:
def Viterbi_Mod1_Rule(words, train_bag = train_tagged_words):
    state = []
    T = sorted(list(set([pair[1] for pair in train_bag])))
     
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                 
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
             
        pmax = max(p)
        state_max = rule_based_tagger.tag([word])[0][1]       
        
         
        if(pmax==0): #For Unknown words using rule based tagger as emission probability will be 0
            state_max = rule_based_tagger.tag([word])[0][1] # assign based on rule based tagger
        else:
            if state_max != 'X':
                # For Known words Assigning state for which probability is maximum
                state_max = T[p.index(pmax)]                
             
        
        state.append(state_max)
    return list(zip(words, state))

#### Evaluating tagging accuracy for Techniquie 1

In [17]:
tagged_seq_mod1 = Viterbi_Mod1_Rule(test_tagged_words)

In [36]:
# accuracy
check1 = [i for i, j in zip(tagged_seq_mod1, test_run_base) if i == j] 

accuracy1 = len(check1)/len(tagged_seq_mod1)

accuracy1

0.9501765398195371

## Technique 2 Consider only Transition Probabaility for Unknown words

In [19]:
def Viterbi_Mod2_Rule(words, train_bag = train_tagged_words):
    state = []
    T = sorted(list(set([pair[1] for pair in train_bag])))
     
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                 
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            
            if emission_p != 0: 
                state_probability = emission_p * transition_p
                p.append(state_probability)
            else: # when word does not exist
                state_probability = 0.00001 * transition_p
                p.append(state_probability)
            
             
                     
        pmax = max(p)
        # getting state for which probability is maximum     
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

#### Evaluating tagging accuracy for Technique 2


In [20]:
tagged_seq_mod2 = Viterbi_Mod2_Rule(test_tagged_words)

In [37]:
# accuracy
check2 = [i for i, j in zip(tagged_seq_mod2, test_run_base) if i == j] 

accuracy2 = len(check2)/len(tagged_seq_mod2)

accuracy2

0.9321302471557473

## Technique 3 Additive Smooting and Assigning Noun (Most Frequent) POS Tag for Unknown Tags Apart from Punctuation

In [22]:
def Viterbi_Mod3_Rule(words, train_bag = train_tagged_words):
    state = []
    T = sorted(list(set([pair[1] for pair in train_bag])))
     
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                 
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            
            lambda_laplace = 0.00001 # Additive smooting
     
            state_probability = (emission_p * transition_p) + lambda_laplace
            
           
            p.append(state_probability)
             
                     
        pmax = max(p)
        # getting state for which probability is maximum     
        state_max = T[p.index(pmax)] 
        state.append(state_max)
        #Creating a df to change the unknown tags to most frequent tags
        data_tuples = list(zip(words, state))
        DF_Mod = pd.DataFrame(data_tuples, columns=['words','state'])
        DF_Mod.loc[(DF_Mod['words'] != ".") & (DF_Mod['words'] != ",") & (DF_Mod['state'] == ".") , 'newstate'] = 'NOUN'
        DF_Mod.loc[(DF_Mod['words'] == ".")  & (DF_Mod['state'] == ".") , 'newstate'] = '.'
        DF_Mod.loc[(DF_Mod['words'] == ",")  & (DF_Mod['state'] == ".") , 'newstate'] = '.'
        DF_Mod.loc[(DF_Mod['state'] != ".") , 'newstate'] = DF_Mod['state']
        DF_Mod = DF_Mod[['words', 'newstate']]
        words1 = DF_Mod['words'].tolist()
        state1 = DF_Mod['newstate'].tolist()

    return list(zip(words1, state1))

#### Evaluating tagging accuracy for Technique 3

In [23]:
tagged_seq_mod3 = Viterbi_Mod3_Rule(test_tagged_words)

In [38]:
# accuracy
check3 = [i for i, j in zip(tagged_seq_mod3, test_run_base) if i == j] 

accuracy3 = len(check3)/len(tagged_seq_mod3)

accuracy3

0.9056492742251864

### Compare the tagging accuracies of the modifications with the vanilla Viterbi algorithm

Vanilla Viterbi  = 90.5 %



Technique 1      = 95 % (Rule Based)


Technique 2      = 93.2 % (Consider only Transition Probabaility for Unknown words)


Technique 3      = 90.5 % (Additive Smooting + Assigning Noun (Most Frequent) POS Tag for Unknown Tags Apart from Punctuation)

### List down cases which were incorrectly tagged by original POS tagger and got corrected by your modifications

In [25]:
## Testing Sentence 1
sentence_test1 = 'Android is a mobile operating system developed by Google.'
words1 = word_tokenize(sentence_test1)


Test11 = Viterbi(words1)
Test12 = Viterbi_Mod1_Rule(words1)
Test13 = Viterbi_Mod2_Rule(words1)
Test14 = Viterbi_Mod3_Rule(words1)

In [26]:
print(Test11)
print(Test12)
print(Test13)
print(Test14)

[('Android', '.'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', '.'), ('.', '.')]
[('Android', 'NOUN'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'NOUN'), ('.', '.')]
[('Android', 'NOUN'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'DET'), ('.', '.')]
[('Android', 'NOUN'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'NOUN'), ('.', '.')]


In [27]:
## Testing Sentence 2
sentence_test2 = 'Android has been the best-selling OS worldwide on smartphones since 2011 and on tablets since 2013.'
words2 = word_tokenize(sentence_test2)


Test21 = Viterbi(words2)
Test22 = Viterbi_Mod1_Rule(words2)
Test23 = Viterbi_Mod2_Rule(words2)
Test24 = Viterbi_Mod3_Rule(words2)

In [28]:
print(Test21)
print(Test22)
print(Test23)
print(Test24)

[('Android', '.'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', '.'), ('worldwide', '.'), ('on', 'ADP'), ('smartphones', '.'), ('since', 'ADP'), ('2011', '.'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', '.'), ('.', '.')]
[('Android', 'NOUN'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('on', 'ADP'), ('smartphones', 'VERB'), ('since', 'ADP'), ('2011', 'NUM'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'NUM'), ('.', '.')]
[('Android', 'NOUN'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('on', 'ADP'), ('smartphones', 'DET'), ('since', 'ADP'), ('2011', 'DET'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'DET'), ('.', '.')]
[('Android', 'NOUN'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-s

In [29]:
## Testing Sentence 3
sentence_test3 = "Google and Twitter made a deal in 2015 that gave Google access to Twitter's firehose."
words3 = word_tokenize(sentence_test3)


Test31 = Viterbi(words3)
Test32 = Viterbi_Mod1_Rule(words3)
Test33 = Viterbi_Mod2_Rule(words3)
Test34 = Viterbi_Mod3_Rule(words3)

In [30]:
print(Test31)
print(Test32)
print(Test33)
print(Test34)

[('Google', '.'), ('and', 'CONJ'), ('Twitter', '.'), ('made', 'VERB'), ('a', 'DET'), ('deal', 'NOUN'), ('in', 'ADP'), ('2015', '.'), ('that', 'DET'), ('gave', 'VERB'), ('Google', '.'), ('access', 'NOUN'), ('to', 'PRT'), ('Twitter', '.'), ("'s", 'VERB'), ('firehose', '.'), ('.', '.')]
[('Google', 'NOUN'), ('and', 'CONJ'), ('Twitter', 'NOUN'), ('made', 'VERB'), ('a', 'DET'), ('deal', 'NOUN'), ('in', 'ADP'), ('2015', 'NUM'), ('that', 'ADP'), ('gave', 'VERB'), ('Google', 'NOUN'), ('access', 'NOUN'), ('to', 'PRT'), ('Twitter', 'NOUN'), ("'s", 'PRT'), ('firehose', 'NOUN'), ('.', '.')]
[('Google', 'NOUN'), ('and', 'CONJ'), ('Twitter', 'NOUN'), ('made', 'VERB'), ('a', 'DET'), ('deal', 'NOUN'), ('in', 'ADP'), ('2015', 'DET'), ('that', 'ADP'), ('gave', 'VERB'), ('Google', 'X'), ('access', 'NOUN'), ('to', 'PRT'), ('Twitter', 'VERB'), ("'s", 'PRT'), ('firehose', 'VERB'), ('.', '.')]
[('Google', 'NOUN'), ('and', 'CONJ'), ('Twitter', 'NOUN'), ('made', 'VERB'), ('a', 'DET'), ('deal', 'NOUN'), ('in', 

sentence_1 = 'Android is a mobile operating system developed by Google.'

sentence_2 = 'Android has been the best-selling OS worldwide on smartphones since 2011 and on tablets since 2013.'

sentence_3 = "Google and Twitter made a deal in 2015 that gave Google access to Twitter's firehose."

#### The Above three sentences has unknown words  from sample test file which Vanilla Viterbi Algorithm was not able tag correctly. Rule based modifed Viterbi has given the best accuracy of 95% and has tagged the above sentences correctly. Technique 2 where we considered only Transition probability for Unknown words has given a accuracy of 93% and tagged above sentences partially correctly. Technique 3 using additive smooting and assigning Most frequent POS Tag to unknown words except punctuation has also been quite effective in handling these sentences though its accuracy of 90.5% which is almost the same as Vanilla Viterbi .